In [2]:
import numpy as np
import pandas as pd
import ast 

In [3]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [4]:
#merge both datasets using title hence 20 + 4 = 23columns
movies=movies.merge(credits,on='title')


In [5]:
#Genres
#id
#keywords
#title
#overview
#cast
#crew
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


In [7]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [8]:
movies.dropna(inplace=True)

In [9]:
movies.duplicated().sum()

0

In [10]:
#'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
#['Action','Adventure','Fantasy','Science Fiction']

In [11]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L


In [12]:
movies['genres']=movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)

In [13]:
def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if (counter!=3):
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [14]:
#using convert 3 so as to obly get top 3 actors from the dataset entry for cast
movies['cast']=movies['cast'].apply(convert3)


In [15]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if(i['job']=='Director'):
            L.append(i['name'])
            break
    return L


In [16]:
movies['crew']=movies['crew'].apply(fetch_director)

In [17]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [18]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['overview']=movies['overview'].apply(lambda x:[i.replace(" ","") for i in x])

In [19]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords'] +movies['cast']+movies['crew'] 

In [20]:
new_Df=movies[['movie_id','title','tags']]

In [21]:
new_Df['tags'] = new_Df['tags'].apply(lambda x: " ".join(x))

C:\Users\Acer\AppData\Local\Temp\ipykernel_21128\1090961883.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_Df['tags'] = new_Df['tags'].apply(lambda x: " ".join(x))


In [22]:
new_Df['tags'] = new_Df['tags'].apply(lambda x:x.lower())

C:\Users\Acer\AppData\Local\Temp\ipykernel_21128\2417645766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_Df['tags'] = new_Df['tags'].apply(lambda x:x.lower())


In [23]:
new_Df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')


In [25]:
vectors=cv.fit_transform(new_Df['tags']).toarray()

In [26]:
import nltk

In [27]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [28]:
def stem(text):
    y=[]
    for i in text.split():
       y.append(ps.stem(i))
    return " ".join(y)


In [29]:
ps.stem('loved')

'love'

In [30]:
new_Df['tags'] = new_Df['tags'].apply(stem)

C:\Users\Acer\AppData\Local\Temp\ipykernel_21128\722620338.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_Df['tags'] = new_Df['tags'].apply(stem)


In [31]:
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
similarity=cosine_similarity(vectors)

In [33]:
def recommend(movie_name):
    movie_index=new_Df[new_Df['title']==movie_name].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_Df.iloc[i[0]].title)


In [34]:
recommend('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin
Batman


In [35]:
recommend('Avatar')

Titan A.E.
Independence Day
Ender's Game
Aliens vs Predator: Requiem
Battle: Los Angeles
